# Run multiple robots for one day

Run an experiment with multiple robots exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type nrobot1day. This exp/run will refer to other exp/runs to specify the environment, the estimator and the scoring function. It will also list the names of the robots, their policies and other information. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint
import gzip as compress
import copy
import pickle

# from WbfExperiment import simulate_1day
from WbfExperimentMultiRobot import simulate_1day_multirobot
from wbf_helper import get_geometry, create_wbfe, create_policy, create_estimator, create_score

from Robot import Robot

In [2]:
# the experiment for the robot run
experiment = "nrobot1day"
run = "two-random-robots"

exp = Config().get_experiment(experiment, run)
pprint(exp)

resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
if resultsfile.exists():
    print(f"Results file already exists:\n{resultsfile}")
    print(f"Delete this file if re-running is desired.")
    raise Exception("Nothing to do.")

# the exp for the environment
exp_env = Config().get_experiment("environment", exp["exp_environment"])
pprint(exp_env)
# the exp for estimator
exp_estimator = Config().get_experiment("estimator", exp["exp_estimator"])
pprint(exp_estimator)
# the exp for the score
exp_score = Config().get_experiment("score", exp["exp_score"])
pprint(exp_score)

# FIXME: implement me
# the exp/runs for the policy for policies
# exp_policy = Config().get_experiment("policy", exp["exp_policy"])
# pprint(exp_policy)
robotspecs = []
for values in exp["robots"]:
    robotspec = {}
    robotspec["name"] = values["name"]
    robotspec["exp-policy"] = Config().get_experiment("policy", values["exp-policy"]) 
    robotspecs.append(robotspec)
pprint(robotspecs)


Loading pointer config file: C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\nrobot1day\two-random-robots_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: nrobot1day/two-random-robots successfully loaded
{'data_dir': WindowsPath('C:/Users/lboloni/Documents/Code/_TempData/WBF/ExperimentData/nrobot1day/two-random-robots'),
 'exp_environment': 'miniberry-30',
 'exp_estimator': 'adaptive-disk',
 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/WBF/WaterberryFarms/experiment_configs/nrobot1day/two-random-robots.yaml'),
 'exp_score': 'weighted_asymmetric',
 'group_name': 'nrobot1day',
 'im_resolution': 10,
 'robots': [{'exp-policy': 'random-waypoint', 'name': 'robot-1

In [3]:
choices = {}
choices["estimator-code"] = create_estimator(exp_estimator, exp_env)
choices["score-code"] = create_score(exp_score, exp_env)

choices["velocity"] = exp["velocity"]
choices["timesteps-per-day"] = exp["timesteps-per-day"]
# choices["timesteps-per-day-override"] = exp["timesteps-per-day-override"]
choices["time-start-environment"] = exp["time-start-environment"]
choices["im_resolution"] = exp["im_resolution"]
choices["results-basedir"] = exp["data_dir"]
choices["action"] = "run-one-day"
choices["typename"] = exp_env["typename"]
wbf, wbfe = create_wbfe(exp_env)
# move ahead to the starting point of the environment
wbfe.proceed(choices["time-start-environment"])
choices["wbf"] = wbf
choices["wbfe"] = wbfe
choices["days"] = 1
get_geometry(choices["typename"], choices)

robots = []
for robotspec in robotspecs:
    # create the robot and set the policy
    robot = Robot(robotspec["name"], 0, 0, 0, env=None, im=None)
    policy = create_policy(robotspec["exp-policy"], exp_env)
    robot.assign_policy(policy)
    robots.append(robot)
choices["robots"] = robots


# 
# This is where we actually calling the simulation
#
results = copy.deepcopy(choices)
simulate_1day_multirobot(results)
print(f"Saving results to: {resultsfile}")
with compress.open(resultsfile, "wb") as f:
    pickle.dump(results, f)

INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_tylcv\env_value_00006.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_tylcv\env_value_00006.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_ccr\env_value_00006.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_ccr\env_value_00006.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Use

loading the geometry and environment from saved data
loading done
Saving results to: C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\nrobot1day\two-random-robots\results.pickle


In [4]:
robots

[<Robot.Robot at 0x262fd9a99d0>, <Robot.Robot at 0x2629eb1c8d0>]